<a href="https://colab.research.google.com/github/dhhieu397/Tensorflow-Certificate/blob/master/Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow_datasets as tfds

In [ ]:
imdb, info = tfds.load('imdb_reviews', with_info=True, as_supervised=True)

In [ ]:
import numpy as np
train_data, test_data = imdb['train'], imdb['test']

In [ ]:
# tạo mảng để lưu các câu
train_sentences =[]
test_sentences =[]
train_labels =[]
test_labels =[]

for s, l in train_data:
  train_sentences.append(str(s.numpy)) # s: các câu
  train_labels.append(l.numpy()) #l: là nhãn

for s, l in test_data:
  test_sentences.append(str(s.numpy)) # s: các câu
  test_labels.append(l.numpy()) #l: là nhãn


In [ ]:
train_sentences[0]

'<bound method _EagerTensorBase.numpy of <tf.Tensor: shape=(), dtype=string, numpy=b"This was an absolutely terrible movie. Don\'t be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie\'s ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor\'s like Christopher Walken\'s good name. I could barely sit through it.">>'

In [ ]:
train_labels[0]

0

In [ ]:
# chuyển nhãn thành numpy array
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

In [ ]:
vocal_size = 10000
embedding_dim = 64
max_length = 140 # độ dài tối đa của 1 câu



In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
tokenizer = Tokenizer(num_words=vocal_size, oov_token="<OOV>")
# build từ điển
tokenizer.fit_on_texts(train_sentences)

In [ ]:
# check index(sự xuất hiện) của từ
tokenizer.word_index

{'<OOV>': 1,
 'the': 2,
 'of': 3,
 'and': 4,
 'a': 5,
 'to': 6,
 'is': 7,
 'br': 8,
 'in': 9,
 'it': 10,
 'i': 11,
 'this': 12,
 'that': 13,
 'numpy': 14,
 'was': 15,
 'as': 16,
 'for': 17,
 'with': 18,
 'movie': 19,
 'but': 20,
 'film': 21,
 "'s": 22,
 'on': 23,
 'you': 24,
 'not': 25,
 'are': 26,
 'his': 27,
 'he': 28,
 'have': 29,
 'be': 30,
 'one': 31,
 'bound': 32,
 'shape': 33,
 'string': 34,
 'method': 35,
 'eagertensorbase': 36,
 'tf': 37,
 'tensor': 38,
 'dtype': 39,
 'all': 40,
 'at': 41,
 'by': 42,
 'they': 43,
 'an': 44,
 'who': 45,
 'so': 46,
 'from': 47,
 'like': 48,
 'her': 49,
 "'t": 50,
 'or': 51,
 'just': 52,
 'there': 53,
 'about': 54,
 'out': 55,
 "'": 56,
 'has': 57,
 'if': 58,
 'some': 59,
 'what': 60,
 'good': 61,
 'more': 62,
 'very': 63,
 'when': 64,
 'she': 65,
 'up': 66,
 'can': 67,
 'b': 68,
 'time': 69,
 'no': 70,
 'even': 71,
 'my': 72,
 'would': 73,
 'which': 74,
 'story': 75,
 'only': 76,
 'really': 77,
 'see': 78,
 'their': 79,
 'had': 80,
 'were': 81,


In [ ]:
# chuuyển câu thành chuỗi số
train_sequences = tokenizer.texts_to_sequences(train_sentences)
# padding train_sequences (các câu)
padded_train_sequences = pad_sequences(train_sequences, maxlen=max_length, truncating='post', padding='post') # cắt và padding đằng sau

In [ ]:
padded_train_sequences.shape

(25000, 140)

In [ ]:
test_sequences = tokenizer.texts_to_sequences(test_sentences)
padded_test_sequences = pad_sequences(test_sequences, maxlen=max_length, truncating='post', padding='post')

In [ ]:
padded_test_sequences.shape

(25000, 140)

In [ ]:
# model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding


In [ ]:
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense


In [ ]:
model = Sequential()
model.add(Embedding(vocal_size, embedding_dim, input_length=max_length))

In [ ]:
model.add(Flatten())

In [ ]:
model.add(Dense(10, activation='relu'))

In [ ]:
# vì là bài toán phân loại nên đưa về lớp dense 1 và activation là sigmoid
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])

In [ ]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 140, 64)           640000    
_________________________________________________________________
flatten_2 (Flatten)          (None, 8960)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 10)                89610     
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 11        
Total params: 729,621
Trainable params: 729,621
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(padded_train_sequences, train_labels, epochs=10, validation_data=(padded_test_sequences, test_labels))

Epoch 1/10
782/782 [==============================] - 9s 11ms/step - loss: 0.4610 - acc: 0.7629 - val_loss: 0.3743 - val_acc: 0.8310
Epoch 2/10
782/782 [==============================] - 8s 11ms/step - loss: 0.1588 - acc: 0.9467 - val_loss: 0.4712 - val_acc: 0.8101
Epoch 3/10
782/782 [==============================] - 9s 11ms/step - loss: 0.0191 - acc: 0.9971 - val_loss: 0.5548 - val_acc: 0.8168
Epoch 4/10
782/782 [==============================] - 8s 10ms/step - loss: 0.0023 - acc: 0.9999 - val_loss: 0.6159 - val_acc: 0.8184
Epoch 5/10
782/782 [==============================] - 8s 11ms/step - loss: 6.9746e-04 - acc: 1.0000 - val_loss: 0.6583 - val_acc: 0.8208
Epoch 6/10
782/782 [==============================] - 8s 11ms/step - loss: 3.3903e-04 - acc: 1.0000 - val_loss: 0.6959 - val_acc: 0.8215
Epoch 7/10
782/782 [==============================] - 9s 11ms/step - loss: 1.8950e-04 - acc: 1.0000 - val_loss: 0.7318 - val_acc: 0.8216
Epoch 8/10
782/782 [==============================] - 9s 

In [ ]:
# predict
test_sen = "I hate this movie"
test_seq = tokenizer.texts_to_sequences(test_sen)
padded_test_seq = pad_sequences(test_seq, maxlen=max_length, truncating="post", padding="post")

In [ ]:
padded_test_seq

array([[  11,    0,    0, ...,    0,    0,    0],
       [   0,    0,    0, ...,    0,    0,    0],
       [2028,    0,    0, ...,    0,    0,    0],
       ...,
       [1931,    0,    0, ...,    0,    0,    0],
       [  11,    0,    0, ...,    0,    0,    0],
       [ 975,    0,    0, ...,    0,    0,    0]], dtype=int32)

In [ ]:
model.predict(padded_test_seq)

array([[0.0986478 ],
       [0.09255391],
       [0.09627596],
       [0.09079346],
       [0.08682999],
       [0.08941242],
       [0.09255391],
       [0.08682999],
       [0.09627596],
       [0.0986478 ],
       [0.09254834],
       [0.09255391],
       [0.08873701],
       [0.09478268],
       [0.09895003],
       [0.0986478 ],
       [0.08941242]], dtype=float32)

In [ ]:
# lâý lớp embedding là lớp đàu tiên
e = model.layers[0]

In [ ]:
weights = e.get_weights()[0]

In [ ]:
weights.shape

(10000, 64)

In [ ]:
import io
out_v = io.open('vecs.tsv','w', encoding='utf-8')
# từ nào đi với vector nào nên cần file để lưu
out_m = io.open('meta.tsv', 'w', encoding='utf-8')

for idx in range(1, vocal_size):
  word = tokenizer.index_word[idx]
  vector = weights[idx]
  out_m.write(word+"\n")
  # chueyern số thành chữ, chữ này nằm trên 1 dòng nên cũng là 1 vector
  out_v.write('\t'.join([str(x) for x in vector]) +"\n")
out_m.close()
out_v.close()

In [ ]:
print(tokenizer.index_word)

{1: '<OOV>', 2: 'the', 3: 'of', 4: 'and', 5: 'a', 6: 'to', 7: 'is', 8: 'br', 9: 'in', 10: 'it', 11: 'i', 12: 'this', 13: 'that', 14: 'numpy', 15: 'was', 16: 'as', 17: 'for', 18: 'with', 19: 'movie', 20: 'but', 21: 'film', 22: "'s", 23: 'on', 24: 'you', 25: 'not', 26: 'are', 27: 'his', 28: 'he', 29: 'have', 30: 'be', 31: 'one', 32: 'bound', 33: 'shape', 34: 'string', 35: 'method', 36: 'eagertensorbase', 37: 'tf', 38: 'tensor', 39: 'dtype', 40: 'all', 41: 'at', 42: 'by', 43: 'they', 44: 'an', 45: 'who', 46: 'so', 47: 'from', 48: 'like', 49: 'her', 50: "'t", 51: 'or', 52: 'just', 53: 'there', 54: 'about', 55: 'out', 56: "'", 57: 'has', 58: 'if', 59: 'some', 60: 'what', 61: 'good', 62: 'more', 63: 'very', 64: 'when', 65: 'she', 66: 'up', 67: 'can', 68: 'b', 69: 'time', 70: 'no', 71: 'even', 72: 'my', 73: 'would', 74: 'which', 75: 'story', 76: 'only', 77: 'really', 78: 'see', 79: 'their', 80: 'had', 81: 'were', 82: 'me', 83: 'well', 84: 'we', 85: 'than', 86: 'much', 87: 'been', 88: 'get', 8